## Import Data

In [4]:
%pip install auto-gptq
%pip install optimum
%pip install bitsandbytes
%pip uninstall torch -y
%pip install torch==2.1
%pip install flash-attn


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Found existing installation: torch 2.1.0
Uninstalling torch-2.1.0:
  Successfully uninstalled torch-2.1.0
Note: you may need to restart the kernel to use updated packages.
  Using cached torch-2.1.0-cp311-cp311-manylinux1_x86_64.whl (670.2 MB)

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)


In [5]:
# import os

# Set the environment variable
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [7]:
torch.cuda.empty_cache()
model_name = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=True, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# # Load model and tokenizer
# model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=False, revision="main")

# # Move the entire model to GPU (cuda:0)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.00s/it]
Some parameters are on the meta device device because they were offloaded to the cpu.


In [8]:
%pip install pandas huggingface_hub


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd

In [10]:
from datasets import load_dataset

In [11]:
data = load_dataset("valurank/News_headlines")

In [12]:
# prompt format
instruction_string = f"""You are HeadlineGPT, an expert in crafting compelling news headlines. Based on the article provided, generate a concise, attention-grabbing headline that accurately reflects the key points and tone of the story, while also maintaining a clear and engaging writing style.

Please respond to the following article.
"""

def fine_tune_template(article, headline):
    return f"<s>[INST] {instruction_string} \n{article} \n[/INST]\n{headline}</s>"

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,DataCollatorWithPadding
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

# Load Tokenizer

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Prepare Model for training

In [15]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [18]:
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.embed_dropout
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.qkv_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.activation_fn
model.layers.0.input_layernorm
model.layers.0.resid_attn_dropout
model.layers.0.resid_mlp_dropout
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.qkv_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.activation_fn
model.layers.1.input_layernorm
model.layers.1.resid_attn_dropout
model.layers.1.resid_mlp_dropout
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.o_proj
model.layers.2.self_attn.qkv_proj
model.layers.2.self_attn.rotary_emb
model.layers.

In [19]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["self_attn.qkv_proj", "self_attn.o_proj"],  # Adjusted to match your model
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 3,825,798,144 || trainable%: 0.1233


In [20]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


# Fine Tune Model

In [21]:
# hyperparameters
lr = 2e-4
batch_size = 2
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "vishals9711_ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

/home/921416519/tinker/headline_finetune/venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
# Define the tokenize function
def tokenize_function(batch):
    # Extract text
    articles = batch['article']
    headlines = batch['headline']
    texts = [fine_tune_template(article=a, headline=h) for a, h in zip(articles, headlines)]

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        texts,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)
# test_tokenized_data = [tokenize_function(item) for item in fine_tune_test_list]
# train_tokenized_data = [tokenize_function(item) for item in fine_tune_test_list]

Map: 100%|██████████| 1200/1200 [00:00<00:00, 4390.64 examples/s]


In [23]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    args=training_args,
    data_collator=data_collator
)


# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You are not running the flash-attention implementation, expect numerical differences.


RuntimeError: Function MmBackward0 returned an invalid gradient at index 1 - expected device meta but got cuda:0

# Push model to HF

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
hf_name = 'shawhin' # your hf username or org name
model_id = hf_name + "/" + "shawgpt-ft"
model.push_to_hub(model_id)
trainer.push_to_hub(model_id)